# Initializing Libraries

In [1]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import time

# Setting Some parameters to web driver to translate web page into english language

In [2]:
options = webdriver.ChromeOptions()
prefs = {
  "translate_whitelists": {"fr":"en","de":"en","it":"en","es":"en"},
  "translate":{"enabled":"true"}
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(r'chromedriver.exe',chrome_options=options)

C:\Users\deep\AppData\Local\Temp/ipykernel_34820/2363375309.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'chromedriver.exe',chrome_options=options)
C:\Users\deep\AppData\Local\Temp/ipykernel_34820/2363375309.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r'chromedriver.exe',chrome_options=options)


In [3]:
data = pd.read_csv("Amazon Scraping.csv")

In [4]:
data.head()

,Unnamed: 0,id,Asin,country
0,0,1,1015,de
1,1,2424796,1015,fr
2,2,2,000004458X,de
3,3,2424797,000004458X,fr
4,4,3,1002198,de


# The below code with go through all the entries given in the excel sheet. After running this code, I found that only 93 links for working correctly. I have saved those links to "links.txt" file.

In [ ]:
count = 0
for i in range(len(data)):
    country = data.at[i,'country']
    asin = data.at[i,'Asin']
    path = "https://www.amazon."+country+"/dp/"+asin
    driver.get(path)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html)
    try:
        print(i,path)
        print(soup.find_all("span", {"id": "productTitle"})[0].text.strip())
        working_urls.append(path)
        count = count + 1
    except:
        print(str(i),"404 Error")

In [5]:
list_products  = []

# The below code will scrap product_title, product_image_link, product_price, product details from those 93 links and save them in list in the form of dictionary.

In [6]:
f = open("links.txt", "r")

for x in f:
    driver.get(x)
    dict_product_details = {}
    dict_product = {}
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html)
    print(x)
    
    print(soup.find_all("span", {"id": "productTitle"})[0].text.strip())
    pt = soup.find_all("span", {"id": "productTitle"})[0].text.strip()
    dict_product['product_title'] = pt
    
    print(soup.find_all("div", {"id": "main-image-container"})[0].img['src'])
    image_link = soup.find_all("div", {"id": "main-image-container"})[0].img['src']
    dict_product['product_image_url'] = image_link
    
    if len(soup.find_all("span", {"class": "tmm-olp-links"})) == 0:
        print(soup.find_all("span", {"class": "a-offscreen"})[0].text)
        product_price = soup.find_all("span", {"class": "a-offscreen"})[0].text
        dict_product['product_price'] = product_price
    elif len(soup.find_all("span", {"class": "olp-new olp-link"})) == 0 and  \
                len(soup.find_all("span", {"class": "olp-used olp-link"})) == 0:
        dict_product['product_price'] = "Not Given"
        print("emtpy")
    else:
        try:
            print(soup.find_all("span", {"class": "olp-new olp-link"})[0].a.text.split())
            if(len(soup.find_all("span", {"class": "olp-new olp-link"})[0].a.text.split()) == 4):
                print(soup.find_all("span", {"class": "olp-new olp-link"})[0].a.text.split()[-1])
                product_price = soup.find_all("span", {"class": "olp-new olp-link"})[0].a.text.split()[-1]
                dict_product['product_price'] = product_price
            else:
                print(''.join(soup.find_all("span", {"class": "olp-new olp-link"})[0].a.text.split()[-2:]))
                product_price = ''.join(soup.find_all("span", {"class": "olp-new olp-link"})[0].a.text.split()[-2:])
                dict_product['product_price'] = product_price
        except:
            print(soup.find_all("span", {"class": "olp-used olp-link"})[0].a.text.split())
            if(len(soup.find_all("span", {"class": "olp-used olp-link"})[0].a.text.split())== 4):
                print(soup.find_all("span", {"class": "olp-used olp-link"})[0].a.text.split()[-1])
                product_price = soup.find_all("span", {"class": "olp-used olp-link"})[0].a.text.split()[-1]
                dict_product['product_price'] = product_price
            else:
                print(''.join(soup.find_all("span", {"class": "olp-used olp-link"})[0].a.text.split()[-2:]))
                product_price = ''.join(soup.find_all("span", {"class": "olp-used olp-link"})[0].a.text.split()[-2:])
                dict_product['product_price'] = product_price
                
    if len(soup.find_all("div", {"id": "detailBullets_feature_div"}))>0:
        if len(soup.find_all("div", {"id": "detailBullets_feature_div"})[0].ul.find_all('li'))>0:
            for i1 in soup.find_all("div", {"id": "detailBullets_feature_div"})[0].ul.find_all('li'):
                string_encode = i1.text.encode("ascii", "ignore")
                string_decode = string_encode.decode()
                #print(" ".join(string_decode.split()))
                dict_product_details[" ".join(string_decode.split()).split(':')[0].strip()] = " ".join(string_decode.split()).split(':')[1].strip()
                dict_product['product_details'] = dict_product_details
        else:
            dict_product['product_details'] = 'Not Given'
            
    elif len(soup.find_all("div", {"id": "prodDetails"}))>0:
        for i1 in soup.find_all("table", {"id": "productDetails_techSpec_section_1"})[0].find_all('tr'):
            string_encode1 = i1.th.text.encode("ascii", "ignore")
            string_decode1 = ' '.join(string_encode1.decode().split())
            string_encode2 = i1.td.text.encode("ascii", "ignore")
            string_decode2 = ' '.join(string_encode2.decode().split())
            dict_product_details[string_decode1] = string_decode2

        for i1 in soup.find_all("table", {"id": "productDetails_detailBullets_sections1"})[0].find_all('tr'):
            string_encode1 = i1.th.text.encode("ascii", "ignore")
            string_decode1 = ' '.join(string_encode1.decode().split())
            string_encode2 = i1.td.text.encode("ascii", "ignore")
            string_decode2 = ' '.join(string_encode2.decode().split())
            dict_product_details[string_decode1] = string_decode2
        dict_product['product_details'] = dict_product_details
    
        #print(dict_product_details)
    list_products.append(dict_product)
    #print(list_products)

https://www.amazon.de/dp/000004458X

Old Spice Shaving Cream - 70 g (original) - Pack of 2
https://m.media-amazon.com/images/I/41xHbjdSXJL._SX355_PIbundle-2,TopRight,0,0_SX355SY222SH20_.jpg
€8.00
https://www.amazon.de/dp/000101742X

Short Story: Violoncello und Klavier. (Cello-Bibliothek)
https://images-na.ssl-images-amazon.com/images/I/41zw0783tpL._SX380_BO1,204,203,200_.jpg
['3', 'New', 'from', '€16.00']
€16.00
https://www.amazon.fr/dp/000101742X

Short story (dushkin/hegyi) violon
https://images-na.ssl-images-amazon.com/images/I/41zw0783tpL._SX380_BO1,204,203,200_.jpg
['1', 'Neuf', 'à', 'partir', 'de', '23,72', '€']
23,72€
https://www.amazon.de/dp/000102163X

Concerto grosso g-Moll: Weihnachtskonzert. op. 6/8. 2 Violinen, Violoncello, Streichorchester und Cembalo. Partitur. (Concertino)
https://images-na.ssl-images-amazon.com/images/I/51sNFyf2t6L._SX378_BO1,204,203,200_.jpg
['2', 'New', 'from', '€18.00']
€18.00
https://www.amazon.fr/dp/000102163X

Concerto grosso 8 g opus 6 partitu


https://www.amazon.de/dp/000109257X

Happy Birthday: Humoristische Variationen über ein Geburtstagslied. Klavier 4-händig. (Edition Schott Einzelausgabe)
https://images-na.ssl-images-amazon.com/images/I/41wLZGTLuGL._SX379_BO1,204,203,200_.jpg
['2', 'New', 'from', '€6.50']
€6.50
https://www.amazon.fr/dp/000109257X

Happy birthday piano
https://images-na.ssl-images-amazon.com/images/I/41wLZGTLuGL._SX379_BO1,204,203,200_.jpg
['1', 'Neuf', 'à', 'partir', 'de', '13,20', '€']
13,20€
https://www.amazon.de/dp/000109453X

6 Duos: op. 1. 2 Flöten. Spielpartitur. (Edition Schott)
https://images-na.ssl-images-amazon.com/images/I/41yNWRfXqDL._SX379_BO1,204,203,200_.jpg
['4', 'New', 'from', '€24.50']
€24.50
https://www.amazon.fr/dp/000109453X

Duos(6) opus 1 flute traversiere
https://images-na.ssl-images-amazon.com/images/I/41yNWRfXqDL._SX379_BO1,204,203,200_.jpg
['1', 'Neuf', 'à', 'partir', 'de', '33,14', '€']
33,14€
https://www.amazon.de/dp/000109825X

Csárdás-Kompositionen: Klarinette und Klavier

https://www.amazon.de/dp/000417240X

Kantate 86 Wahrlich, Wahrlich
https://images-na.ssl-images-amazon.com/images/I/41aWBN0BW1L._SX350_BO1,204,203,200_.jpg
['4', 'New', 'from', '€6.50']
€6.50
https://www.amazon.fr/dp/000417240X

Kantate 86 wahrlich, wahrlich
https://images-na.ssl-images-amazon.com/images/I/41aWBN0BW1L._SX350_BO1,204,203,200_.jpg
emtpy
https://www.amazon.de/dp/000417268X

Kantate 114 Ach Lieben
https://images-na.ssl-images-amazon.com/images/I/41mIiAzCWsL._SX349_BO1,204,203,200_.jpg
['5', 'New', 'from', '€8.32']
€8.32
https://www.amazon.fr/dp/000417268X

Kantate 114 ach lieben
https://images-na.ssl-images-amazon.com/images/I/41mIiAzCWsL._SX349_BO1,204,203,200_.jpg
['1', 'Neuf', 'à', 'partir', 'de', '9,00', '€']
9,00€
https://www.amazon.de/dp/000417271X

Kantate 117 Sie Lob Und Ehr
https://images-na.ssl-images-amazon.com/images/I/41hlFmvlMsL._SX351_BO1,204,203,200_.jpg
['3', 'New', 'from', '€7.90']
€7.90
https://www.amazon.fr/dp/000417271X

Kantate 117 sie lob und ehr
htt

In [7]:
import json
with open("output.json", "w") as final:
    json.dump(list_products, final)

In [8]:
len(list_products)

93

# Saving Data to Postgres sql : elephantSQL.com

In [9]:
from sqlalchemy import create_engine
import psycopg2
POSTGRES_ADDRESS = 'lallah.db.elephantsql.com'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'sppbwxsk' 
POSTGRES_PASSWORD = 'BhzVquqckuEN-jwI9x1kHiUGCx3XYTZc'
POSTGRES_DBNAME = 'sppbwxsk' 

In [10]:
postgres_str='postgresql://'+POSTGRES_USERNAME+':'+POSTGRES_PASSWORD+'@'+POSTGRES_ADDRESS+':'+POSTGRES_PORT+'/'+POSTGRES_DBNAME

In [11]:
engine = psycopg2.connect(postgres_str)

In [12]:
cur = engine.cursor()

In [13]:
sql =("""CREATE TABLE productss(
   title VARCHAR(255),
   image_link VARCHAR(255),
   price VARCHAR(255),
   details VARCHAR(1000)
)""")

In [14]:
cur.execute(sql)
engine.commit()

In [63]:
import json
f = open('output.json')
data = json.load(f)

In [64]:
for i in range(len(data)):
    text = ''
    for i1 in data[i]['product_details'].keys():
        text += i1 + ' : ' + data[i]['product_details'][i1]+', '
    postgres_insert_query = '''INSERT INTO productss(title, image_link, price, details) VALUES (%s,%s,%s,%s)'''
    record_to_insert = (data[i]['product_title'],data[i]['product_image_url'], data[i]['product_price'], text)
    cur.execute(postgres_insert_query, record_to_insert)
engine.commit()